In [ ]:
import pandas as pd
from pingouin import intraclass_corr
from tqdm.notebook import tqdm
import numpy as np
import warnings

warnings.filterwarnings("ignore", message="divide by zero encountered in scalar divide")

file_path = r"\PATH.csv"
radiomics_data = pd.read_csv(file_path)

start_col = "original_shape_Elongation"
end_col = "lbp-3D-k_ngtdm_Strength"
exclude_cols = [
    "lbp-3D-k_ngtdm_Strength",
    "lbp-3D-m2_firstorder_Minimum",
    "lbp-3D-m1_firstorder_Minimum"
]

radiomic_features = [
    col for col in radiomics_data.loc[:, start_col:end_col].columns
    if col not in exclude_cols
]

id_vars = ["Case", "Test", "Segment ID"]
melted_data = radiomics_data.melt(
    id_vars=id_vars, 
    value_vars=radiomic_features, 
    var_name="Feature", 
    value_name="Value"
)

# Add a unique identifier for each segment within the same case
melted_data["Group"] = melted_data["Case"] + "_" + melted_data["Segment ID"]

# Compute ICC (3,1) for all numeric features 
icc_results = []
for feature in tqdm(radiomic_features, desc="Computing ICC"):
    feature_data = melted_data[melted_data["Feature"] == feature]
    icc_data = feature_data.pivot(index="Group", columns="Test", values="Value")
    if icc_data.shape[1] == 2:  
        icc_data = icc_data.dropna()
        icc_data = icc_data.reset_index()
        icc_data = icc_data.melt(id_vars="Group", var_name="Test", value_name="Value")
        icc = intraclass_corr(
            data=icc_data,
            targets="Group",
            raters="Test",
            ratings="Value"
        ).query('Type == "ICC3"')
        icc_results.append([
            feature, 
            icc.iloc[0].ICC, 
            icc.iloc[0]["CI95%"], 
            icc.iloc[0]["pval"]
        ])

icc_df = pd.DataFrame(icc_results, columns=["Feature", "ICC (3,1)", "95% CI", "p-value"])

median_icc = icc_df["ICC (3,1)"].median()
percentile_25 = np.percentile(icc_df["ICC (3,1)"], 25)
percentile_75 = np.percentile(icc_df["ICC (3,1)"], 75)

print(f"Median ICC: {median_icc:.5f} (25th percentile: {percentile_25:.5f}, 75th percentile: {percentile_75:.5f})")
icc_df#.to_csv(r'19F_radiomics_ICC_results.xlsx', index=False)